In [1]:
import pandas as pd
import numpy as np
from linestar import get_proj_data, get_historical_data

import seaborn as sns

In [193]:
data = get_proj_data()

/home/eadains/DFS/linestar.py:53: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 53 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


AttributeError: 'NoneType' object has no attribute 'get'

In [2]:
data = get_historical_data("2022-05-19")

/home/eadains/DFS/linestar.py:53: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 53 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


# Correlation Matrix

In [4]:
bat_ord_corr = pd.read_csv("./data/batting_order_corr.csv", index_col="Order")
bat_ord_corr.columns = bat_ord_corr.columns.astype(float)
opp_pitcher_corr = pd.read_csv("./data/opp_pitcher_corr.csv", index_col=0)
opp_pitcher_corr = opp_pitcher_corr.loc["Scored", "Scored Opposing"]

corr = pd.DataFrame(columns=data["Player"], index=data["Player"], dtype=float)

In [6]:
for row in data.itertuples():
    # Correlation with themselves is 1
    corr.loc[row.Player, row.Player] = 1

    # If pitcher, set correlation to everyone else to 0
    if row.Position == "P":
        corr.loc[row.Player, corr.columns != row.Player] = 0

    else:
        # Setting correlation to other batters on the same team according to
        # batting order
        for teammate in data.loc[data["Team"] == row.Team, :].itertuples():
            # If the teammate is the pitcher, then 0 correlation
            if teammate.Position == "P":
                corr.loc[row.Player, teammate.Player] = 0
                corr.loc[teammate.Player, row.Player] = 0
            else:
                order_corr = bat_ord_corr.loc[row.Order, teammate.Order]
                corr.loc[row.Player, teammate.Player] = order_corr
                corr.loc[teammate.Player, row.Player] = order_corr

        # Set correlation to opposing pitcher
        corr.loc[row.Player, row.Opp_Pitcher] = opp_pitcher_corr
        corr.loc[row.Opp_Pitcher, row.Player] = opp_pitcher_corr
        # Correlations with every other player is 0
        corr.loc[row.Player, corr.loc[row.Player].isna()] = 0

In [7]:
# Check that the correlation matrix is symmetric and all its eigvenvalues are >= 0
# These two conditions jointly imply the matrix is positive semi-definite
np.array_equal(corr, corr.T) & np.all(np.linalg.eigvals(corr) >= 0)

True

# Standard Deviation

In [53]:
historical = pd.read_csv("./data/linestar_data.csv")
hist_std = historical.groupby("Player").std()["Scored"]

# Default standard deviations for players with missing values
default_pitcher_std = 15
default_other_std = 10

# Get historical standard deviation of scored points for players
# on the current slate
hist_std = hist_std.loc[data["Player"]]

for player in hist_std[hist_std.isna()].index:
    player_position = data.loc[data["Player"] == player, "Position"].values[0]
    if player_position == "P":
        hist_std.loc[player] = default_pitcher_std
    else:
        hist_std.loc[player] = default_other_std

# Score Distribution

In [73]:
cov = np.diag(hist_std) @ corr @ np.diag(hist_std)